# Identifying Data Cleaning and Preprocessing Needs

## Ideation

1. Dataset Ingestion & Exploration
* Allow users to upload or provide a dataset sample.
* Automatically check for:
* Missing values
* Data types of columns
* Cardinality of categorical features
* Distribution of numerical features
* Outliers (IQR or Z-score)
2. Preprocessing Needs Detection
* Identify necessary transformations:
* Handling missing values (imputation strategies)
* Encoding categorical variables (One-Hot, Label Encoding)
* Normalization or Standardization for numerical features
* Feature engineering suggestions
* Handling skewed distributions
3. Brief Statistical Analysis
* Calculate summary statistics:
* Mean, Median, Mode, Variance, Standard Deviation
* Correlations
* Feature importance (if applicable)
* Generate insights about dataset structure
4. ML Algorithm Suggestions
* Determine if the dataset is:
* Supervised: Classification or Regression
* Unsupervised: Clustering, Dimensionality Reduction, or Association Rules
* Based on feature types and dataset size, recommend:
* Classical ML models (Logistic Regression, Decision Trees, SVM, Random Forest, etc.)
* Deep Learning (if appropriate)
* Explain why certain models might perform well
5. Visualization Suggestions
* Recommend suitable visualizations:
* Histograms, box plots for distributions
* Heatmaps for correlations
* Scatter plots for relationships
* PCA/T-SNE for high-dimensional data visualization
6. Project Ideas
* Suggest potential use cases based on dataset type:
* If financial data → Fraud detection, Customer segmentation
* If text data → Sentiment Analysis, Topic Modeling
* If time-series → Forecasting, Anomaly Detection


---

# Part 1. Dataset Ingestion & Exploration

In [122]:
# Load needed libraries
import pandas as pd
import numpy as np
import os

import re


In [123]:
os.chdir("/Users/ghizlanerehioui/Desktop/Capstone/Data Files/Automobile")
df = pd.read_csv("Automobile_data.csv")

In [124]:
df.head()

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450


In [127]:
#####
# Create a table summarizing this as needed
#####

In [139]:
"""
# info()
df.info()

# A lot of columns are object b/c of an incorrect value: in this case "?" 
# We need to generalize this for other datasets - not necessarily the same 


# Checking for unique values within each column
for col_name in col_names:
    print(f"Unique values in the feature: '{col_name}:'")
    print(df[col_name].unique())
    print(f"Count of unique values in the feature: '{col_name}:'")
    print(df[col_name].value_counts())
    print("\n")
    
"""

'# info()\ndf.info()\n\n# A lot of columns are object b/c of an incorrect value: in this case "?" \n# We need to generalize this for other datasets - not necessarily the same \n\n\n# Checking for unique values within each column\nfor col_name in col_names:\n    print(f"Unique values in the feature: \'{col_name}:\'")\n    print(df[col_name].unique())\n    print(f"Count of unique values in the feature: \'{col_name}:\'")\n    print(df[col_name].value_counts())\n    print("\n")'

In [ ]:
# df.shape
print(f"Checking for duplicates:\n")

num_rows_before_dupdrop = df.shape[0]
num_cols_before_dupdrop= df.shape[1]
print(f"The dataset initially contains {num_rows_before_dupdrop} rows and {num_cols_before_dupdrop} columns (features).\n")

# Duplicate rows/columns
len_dup_rows = len(df[df.duplicated()])
len_dup_cols = len(df.T[df.T.duplicated()])
print(f"""
      - The number of duplicate rows in this dataset is {len_dup_rows}.
      - The number of duplicate columns in this dataset is {len_dup_cols}.
      """)

if len(df.duplicated()) == 0:
      print(f"""
      - The duplicate rows in this dataset are {df[df.duplicated()]}.""")
      df.drop_duplicates(inplace=True)

if len(df.T.duplicated()) == 0:
      print(f"""
      - The duplicate columns in this dataset are {df.T[df.T.duplicated()]}.""")
      df.T.drop_duplicates(inplace=True)

num_rows = df.shape[0]
num_cols = df.shape[1]

if len_dup_cols*len_dup_rows != 0:
      print(f"- After dropping duplicated rows and columns, the dataset contains {num_rows} rows and {num_cols} columns (features).\n")
else:
      print(f"--> There are no duplicates to drop!\n")


Checking for duplicates:

The dataset initially contains 205 rows and 26 columns (features).


      - The number of duplicate rows in this dataset is 0.
      - The number of duplicate columns in this dataset is 0.
      
--> There are no duplicates to drop!



In [153]:
col_names = df.columns
print(f"There are {len(col_names)} colummns in the dataset.\nThe column names are (features):")
for col_name in col_names:
    print("- "+col_name)
print("\n")

There are 26 colummns in the dataset.
The column names are (features):
- symboling
- normalized-losses
- make
- fuel-type
- aspiration
- num-of-doors
- body-style
- drive-wheels
- engine-location
- wheel-base
- length
- width
- height
- curb-weight
- engine-type
- num-of-cylinders
- engine-size
- fuel-system
- bore
- stroke
- compression-ratio
- horsepower
- peak-rpm
- city-mpg
- highway-mpg
- price




In [161]:
print(f"Looking at each feature: \n\n")

cols_data_inconsis = {}

cols_incorrect_vals_to_fix = {}

for col_name in col_names:
    
    print(f"- The feature: '{col_name}'")
    print(f"- Datatype: {df[col_name].dtypes}")
    
    # Printing the unique elements
    #print(df[col_name].unique()) 
    unique_col_vals = df[col_name].unique().tolist()

    print("- Potential Data Inconsistencies:")

    # if data type is not int or float:
    if df[col_name].dtypes not in ["float64", "float32", "int64", "int32"] :

        # Column contains numbers but there are some incorrect values 
        pattern = re.compile(r"-?\d+(\.\d+)?")
        match_pattern = [bool(pattern.fullmatch(str(val))) for val in unique_col_vals]
        #print(match_pattern)
        
        if any(match_pattern) : 
            # Since the data type is not int nor float, there is at least one incorrect value 
            print("""\t- This feature contains numbers, either an integer or a float. Since the code is unable to catch that the data type is neither an integer nor a float, there is at least one incorrect value:""")
            
            incorrect_vals = list(filter(lambda x: not pattern.fullmatch(str(x)),unique_col_vals))
            print(f"""\t- The incorrect values are: {incorrect_vals}""")
            
            # Store in dict format the incorrect values and the column name correponding to those
            cols_incorrect_vals_to_fix[col_name] = incorrect_vals
        
    # Missing Values
    if df[col_name].isnull().sum() != 0:
        print(f"""\t- {df[col_name].isnull().sum()} is the number of records with missing values in the feature '{col_name}.'""")
        df[col_name].isnull().sum() #df.isnull().sum(axis=0)

    print("\n")

print(f"The columns/features and values to fix in the entire dataset are: {cols_incorrect_vals_to_fix}")

Looking at each feature: 


- The feature: 'symboling':
- Datatype: int64
- Potential Data Inconsistencies:


- The feature: 'normalized-losses':
- Datatype: object
- Potential Data Inconsistencies:
	- This feature contains numbers, either an integer or a float. Since the code is unable to catch that the data type is neither an integer nor a float, there is at least one incorrect value:
	- The incorrect values are: ['?']


- The feature: 'make':
- Datatype: object
- Potential Data Inconsistencies:


- The feature: 'fuel-type':
- Datatype: object
- Potential Data Inconsistencies:


- The feature: 'aspiration':
- Datatype: object
- Potential Data Inconsistencies:


- The feature: 'num-of-doors':
- Datatype: object
- Potential Data Inconsistencies:


- The feature: 'body-style':
- Datatype: object
- Potential Data Inconsistencies:


- The feature: 'drive-wheels':
- Datatype: object
- Potential Data Inconsistencies:


- The feature: 'engine-location':
- Datatype: object
- Potential Data In

In [ ]:
# A more concise way of displaying results / summarizing / ... for prompting


print(f"The columns/features and values to fix in the entire dataset are: {cols_incorrect_vals_to_fix}")

In [ ]:
### How much % data is missing or incorrect

In [ ]:
# Replace the incorrect values and missing values by __________


# Fixing data types
# Fix the datatype as needed -- which columns need to be changed from object to int or float
# if all values except incorrect values are numbers --> change data type


In [ ]:
# New summary:

# There are _____ int, _____ float, _____ string... datatypes in the dataset

## Data Quality Report

## Statistical Analysis

In [131]:
# Central tendenceies / IQR...
df.describe()

,symboling,wheel-base,length,width,height,curb-weight,engine-size,compression-ratio,city-mpg,highway-mpg
count,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000,205.000000
mean,0.834146,98.756585,174.049268,65.907805,53.724878,2555.565854,126.907317,10.142537,25.219512,30.751220
std,1.245307,6.021776,12.337289,2.145204,2.443522,520.680204,41.642693,3.972040,6.542142,6.886443
min,-2.000000,86.600000,141.100000,60.300000,47.800000,1488.000000,61.000000,7.000000,13.000000,16.000000
25%,0.000000,94.500000,166.300000,64.100000,52.000000,2145.000000,97.000000,8.600000,19.000000,25.000000
50%,1.000000,97.000000,173.200000,65.500000,54.100000,2414.000000,120.000000,9.000000,24.000000,30.000000
75%,2.000000,102.400000,183.100000,66.900000,55.500000,2935.000000,141.000000,9.400000,30.000000,34.000000
max,3.000000,120.900000,208.100000,72.300000,59.800000,4066.000000,326.000000,23.000000,49.000000,54.000000


In [132]:
# Correlations
